# Lab02

## 영화 리뷰 분류: 이진 분류 예제

가장 널리 적용되는 이진분류에 대한 문제를 다뤄보도록 하겠습니다.  
이번 예제에서는 Keras를 사용하여 리뷰 텍스트를 기반으로 영화 리뷰를 긍정(positive)와 부정(negative)으로 분류하는 학습을 진행합니다. 

In [ ]:
#path 관련 라이브러리
import os
from os.path import join, isdir

# Scientific Math 라이브러리  
import numpy as np
from sklearn.model_selection import train_test_split

# Visualization 라이브러리
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers, models, optimizers, Input
from keras.utils import to_categorical
from keras.preprocessing import sequence

### IMDB 데이터셋

인터넷 영화 데이터베이스(Internet Moive Database)로부터 가져온 양극단의 리뷰 5만 개로 이루어진 IMDB 데이터셋을 사용하도록하겠습니다.  
이 데이터셋은 훈련 데이터 25,000개와 테스트 데이터 25,000개로 나뉘어 있으며 각각 50%는 부정, 50%는 긍정 리뷰로 구성되어 있습니다.  
  이 데이터는 전처리가 되어 있어 각 리뷰(단어 시퀀스)가 숫자 시퀀스로 변환되어 있습니다. 여기서 각 숫자는 사전에 있는 고유한 단어를 말합니다.

In [ ]:
def load_data(path, num_words=None, skip_top=0, seed=113):
    with np.load(path,allow_pickle=True) as f:
        x_train, labels_train = f['x_train'], f['y_train']
        x_test, labels_test = f['x_test'], f['y_test']

    np.random.seed(seed)
    
    indices = np.arange(len(x_train))
    np.random.shuffle(indices)
    x_train = x_train[indices]
    labels_train = labels_train[indices]
    
    indices = np.arange(len(x_test))
    np.random.shuffle(indices)
    x_test = x_test[indices]
    labels_test = labels_test[indices]
    
    xs = np.concatenate([x_train, x_test])
    labels = np.concatenate([labels_train, labels_test])
    
    if not num_words:
        num_words = max([max(x) for x in xs])

    xs = [[w for w in x if skip_top <= w < num_words] for x in xs]
    
    idx = len(x_train)
    x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
    x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])
    
    return (x_train, y_train), (x_test, y_test)

### 데이터 준비

신경망에는 숫자 리스트를 주입할 수는 없습니다. 리스트를 텐서로 바꾸는 두가지 방법이 있습니다.

> 1. 같은 길이가 되도록 리스트에 패딩(padding)을 추가하고 (samples, sequence_length) 크기의 정수 텐서로 변환합니다.  
> *가장 긴 리뷰는 2,484개의 단어로 이루어져 있으므로 훈련 데이털르 변환한 텐서의 크기는 (25000,2494)가 됩니다.*  
>  그다음 이 정수 텐서를 다룰 수 있는 층을 신경망의 첫 번째 층으로 사용합니다.
>    
>
> 2. 리스트를 원-핫 인코딩(one-hot encoding)하여 0과 1의 벡터로 변환합니다.  
> 그다음 부동 소수 벡터 데이터를 다룰 수 있는 Dense 층을 신경망의 첫번째 층으로 사용합니다.

이번 예제를 통해 두번째 방식을 사용하며 직접 데이터를 원-핫 벡터로 만들어봅니다.

In [ ]:
(train_data, train_labels), (test_data, test_labels) = load_data('./data/imdb.npz', num_words=10000)

In [ ]:
def vectorize_sequences(sequences,dimension=10000):
    results = np.zeros((len(sequences),dimension))
    #크기가(len(sequnces),dimension)이고 모든 원소가 0인 행렬을 만듭니다.
    for i, sequence in enumerate(sequences):
        results[i,sequence] =1. #results[i]에서 특정 인덱스의 위치를 1로 만듭니다.
    return results

x_train = vectorize_sequences(train_data) #훈련 데이터를 벡터로 변환합니다.
x_test = vectorize_sequences(test_data)   #테스트 데이터를 벡터로 변환합니다.

## 레이블은 쉽게 벡터로 바꿀 수 있습니다.
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

이제 신경망에 주입할 데이터가 준비 되었습니다.

### 신경망 모델 만들기

다음은 신경망의 케라스 구현입니다. 이전에 보았던 예제들을 참고하여 직접 신경망을 구현해보겠습니다.  
  
  - 입력 데이터가 벡터이고 레이블은 스칼라(0 또는 1)입니다. 이런 문제에 잘 작동하는 네트워크 종류는 relu활성화 함수를 사용한 완전 연결 층(Dense(n,activation='relu))을 그냥 쌓은 것입니다.  
  Dense에 전달한 매개변수 n은 **은닉 유닛(hidden unit)** 의 개수입니다.  
  하나의 은닉 유닛은 층이 나타내는 표현 공간에서 하나의 차원이 됩니다.  
  은닉 유닛을 늘리면 (표현 공간을 더 고차원으로 만들면) 신경망이 더욱 복잡한 표현을 학습할 수 있지만 계산 비용이 커지고 원하지 않는 패턴을 학습할 수도 있습니다.  

 - Dense 층을 쌓을 때 두 가지 중요한 구조상의 결정이 필요합니다.
 > 얼마나 많은 층을 사용할 것인가?  
 > 각 층에 얼마나 많은 은닉 유닛을 둘 것인가?  
 
 - 중간에 있는 은닉 층은 활성화 함수로 relu를 사용하고 마지막 층은 확률(0과 1 사이의 점수로, 어떤 샘플이 타깃이 '1'일 가능성이 높다는 것은 그 리뷰가 긍정일 가능성이 높다는 것을 의미합니다)을 출력하기 위해 시그모이드 활성화 함수를 사용합니다.  
 
 relu는 음수를 0으로 만드는 함수입니다. 시그모이드는 임의의 값을 [0,1]사이로 압축하므로 출력 값을 확률처럼 해석할 수 있습니다,

In [ ]:
# 모델 정의하기
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#### 손실 함수와 옵티마이저를 선택해야 합니다.
이진 분류 문제고 신경망의 출력이 확률이기 때문에, binary_crossentropy 손실이 적합합니다. 이 함수 뿐만아니라 mean_squared_error도 사용할 수 있습니다. 확률을 출력하는 모델을 사용할 때는 크로스엔트로피가 최선의 선택이라고 합니다.  
**크로스엔트로피(cross entropy)** 는 정보 이론 분야에서 온 개념으로 확률 분포 간의 차이를 측정합니다. 여기에서는 원본 분포와 예측 분포 사이를 측정합니다.

In [ ]:
# 모델 컴파일 하기
model.compile(optimizer = 'rmsprop',loss='binary_crossentropy',metrics=["accuracy"])

케라스에 rmspop, binary_crossentropy, accuracy 가 포함되어 있기 때문에 옵티마이저, 손실 함수, 측정 지표를 문자열로 지정하는 것이 가능합니다. 이따금 옵티마이저의 매개변수를 바꾸거나 자신만의 손실 함수,측정 함수를 전달해야 할 경우가 있습니다.

In [ ]:
# 옵티마이저 설정하기
model.compile(optimizer= optimizers.RMSprop(lr=0.001),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

### 훈련 검증

훈련하는 동안 처음 본 데이터에 대한 모델의 정확도를 측정하기 위해서는 원본 훈련 데이터에서 10,000의 샘플을 떼어 검증 세트를 만들어야 합니다.


In [ ]:
# validation set 준비하기
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

이제 모델을 512개의 샘플씩 미니 배치를 만들어 20번의 에포크(epoch) 동안 훈련시킵니다. 동시에 따로 떼어놓은 샘플에서 손실과 정확도를 측정합니다.  
이렇게 하기 위해서는 validation_data 매개변수에 검증 데이터를 전달해야 합니다.

In [ ]:
# 모델 훈련하기
model.compile(optimizer= 'rmsprop',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

history = model.fit(partial_x_train, partial_y_train,
                   epochs=20, batch_size = 512, validation_data=(x_val,y_val), verbose=2)

In [ ]:
#훈련(train)과 검증(validation) 손실(loss) 그리기

history_dict = history.history
loss= history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# 훈련과 검증 정확도 그리기

plt.clf()  #그래프 초기화
acc = history_dict['acc']
val_acc = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label= 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

손실, 정확도 그래프를 바탕으로 과대적합을 방지하기 위해 에포크를 몇번으로 줄 지 생각할 수 있습니다.  
나온 결과를 바탕으로 epoch 설정을 다시하여 모델을 처음부터 다시 훈련해봅니다.

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16,activation='relu',input_shape=(10000,)))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(optimizer= 'rmsprop',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])
model.fit(x_train, y_train, epochs = 4, batch_size = 512, verbose=2)
results = model.evaluate(x_test, y_test)

In [ ]:
results

### 훈련된 모델로 새로운 데이터에 대해 예측하기

모델을 훈련시킨 후에 predict 메서드를 사용해서 어떤 리뷰가 긍정일 확률을 예측할 수 있습니다.

In [ ]:
model.predict(x_test)